In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as graph
import seaborn as sns
import cartopy as ccrs
import cartopy.crs as ccrs

from tqdm import tqdm
import swifter

from osgeo import ogr, osr
from joblib import Parallel, delayed

tqdm.pandas()
import xarray as xr
from global_land_mask import globe
import geopandas as gpd

from datetime import date
today_date = date.today().strftime("%b_%d").lower()
import scipy.stats as stats

In [100]:
# Species dictionay

species_name_df = pd.read_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/species_name_dictionary.csv')

dict_species_name = species_name_df.set_index('common_name').to_dict()['scientific_name']

# Ontario

In [38]:
lat_long_precision = 3

### After 2007, Broad Scale Monitoring

In [39]:
df_location = pd.read_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Ontario/York_request_ONBSM_Location.csv')
df_location.drop(columns=['Target_species', 'Lake_selection'], inplace=True)
df_location['year'] = pd.DatetimeIndex(pd.to_datetime(df_location['Survey_year_month'])).year

df_location['lat_long'] = (df_location['Lat'].round(lat_long_precision).astype(str) + '_' + 
                           df_location['Long'].round(lat_long_precision).astype(str))
print(f'Location size: {len(df_location)}')

df_variables = df_location[['year', 'lat_long', 'Area_ha', 'Depth_Max', 'Depth_Mn', 'SDF', 'Secchi_Su', 
                            'Thermo_Obs', 'Thermo_Pred', 'TotalPhosphorus(ugL)', 'TrueColour_pctl', 'pH_pctl', 
                            'Conductivity(uScms)', 'TDS (mg/L)', 'DD5_8110', 'AirTemp_8110']]

df_variables = df_variables.groupby(['year', 'lat_long']).mean().reset_index()
print(f'Variables size: {len(df_variables)}')
display(df_variables.head())

Location size: 1409
Variables size: 1406


,year,lat_long,Area_ha,Depth_Max,Depth_Mn,SDF,Secchi_Su,Thermo_Obs,Thermo_Pred,TotalPhosphorus(ugL),TrueColour_pctl,pH_pctl,Conductivity(uScms),TDS (mg/L),DD5_8110,AirTemp_8110
0,2007.0,44.584_-78.844,4764.0,12.8,5.0,2.984901,NaN,NaN,9.770048,NaN,NaN,NaN,NaN,NaN,1975.0,6.4
1,2007.0,48.579_-90.419,3407.0,54.9,18.1,3.101563,NaN,NaN,13.763188,8.2,12.0,38.0,66.0,43.9560,1453.0,2.4
2,2007.0,49.111_-91.881,8467.0,56.4,22.1,5.469126,NaN,11.0,15.693298,5.2,50.0,40.0,25.3,16.8498,1615.0,2.8
3,2008.0,43.057_-81.176,233.0,12.2,3.6,2.478946,1.2,NaN,6.269517,52.5,64.0,64.0,549.0,365.6340,2246.0,8.1
4,2008.0,44.172_-78.855,6650.0,6.1,1.3,5.075877,0.5,NaN,NaN,10.1,33.0,100.0,390.0,259.7400,2065.0,7.2


In [40]:
df_location = df_location[['year', 'lat_long', 'Lat', 'Long', 
                           'BsM_Cycle', 'FMZ', 'Wby_LID', 'Wby_Name']].merge(df_variables, on=['year', 'lat_long'])
display(df_location.head())
print(len(df_location))

,year,lat_long,Lat,Long,BsM_Cycle,FMZ,Wby_LID,Wby_Name,Area_ha,Depth_Max,...,Secchi_Su,Thermo_Obs,Thermo_Pred,TotalPhosphorus(ugL),TrueColour_pctl,pH_pctl,Conductivity(uScms),TDS (mg/L),DD5_8110,AirTemp_8110
0,2012.0,54.146_-85.04,54.14639,-85.03972,1.0,1.0,16-6284-60018,Pine L.,299.0,12.66,...,1.9,11.0,8.846716,12.8,100.0,100.0,137.0,91.242,1069.0,-1.8
1,2012.0,54.165_-85.689,54.16500,-85.68917,1.0,1.0,16-5856-60025,Shamattawa L.,963.0,7.20,...,1.2,NaN,NaN,11.5,100.0,50.0,115.0,76.590,1073.0,-1.8
2,2012.0,54.334_-85.014,54.33444,-85.01361,1.0,1.0,16-6292-60223,Spruce L.,1082.0,16.00,...,2.0,NaN,10.110772,9.5,50.0,100.0,133.0,88.578,1059.0,-1.9
3,2011.0,52.579_-91.541,52.57861,-91.54139,1.0,2.0,15-5988-58262,Windigo L.,8028.0,7.00,...,1.6,NaN,NaN,11.3,63.0,63.0,108.0,71.928,1329.0,-0.3
4,2011.0,52.91_-87.384,52.90972,-87.38417,1.0,2.0,16-4753-58628,Winisk L.,25815.0,16.50,...,3.0,NaN,12.043536,9.0,50.0,88.0,149.0,99.234,1179.0,-1.2


1407


In [41]:
bsm_lakes = df_location[['lat_long', 'Lat', 'Long', 'Wby_Name']]

bsm_lakes.rename(columns = {'Lat': 'latitude', 'Long': 'longitude', 'Wby_Name': 'lake_name'}, inplace=True)
print(len(bsm_lakes))
bsm_lakes.drop_duplicates(inplace=True)
print(len(bsm_lakes))
display(bsm_lakes.head())

1407
1405


/home/arshad/miniconda3/lib/python3.7/site-packages/pandas/core/frame.py:4449: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/arshad/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,lat_long,latitude,longitude,lake_name
0,54.146_-85.04,54.14639,-85.03972,Pine L.
1,54.165_-85.689,54.16500,-85.68917,Shamattawa L.
2,54.334_-85.014,54.33444,-85.01361,Spruce L.
3,52.579_-91.541,52.57861,-91.54139,Windigo L.
4,52.91_-87.384,52.90972,-87.38417,Winisk L.


### AHI Data

In [42]:
df = pd.read_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Ontario/Used/Ontario lakes and fish database_October2013.csv')

ahi_data = df[['Latitude', 'Longitude', 'Lake name']]
ahi_data.columns = ahi_data.columns.str.lower().str.replace(' ', '_')
ahi_data['lat_long'] = (ahi_data['latitude'].round(lat_long_precision).astype(str) + 
                        '_' + ahi_data['longitude'].round(lat_long_precision).astype(str))

print(len(ahi_data))
ahi_data.drop_duplicates(inplace=True)
print(len(ahi_data))
display(ahi_data.head())

9885
9823


/home/arshad/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/arshad/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,latitude,longitude,lake_name,lat_long
0,50.337778,-93.404444,Aerobus L.,50.338_-93.404
1,50.026389,-93.154722,Affleck L.,50.026_-93.155
2,49.459444,-92.468611,Aiabewatik L.,49.459_-92.469
3,50.028889,-92.952500,Little Amesdale L.,50.029_-92.952
4,50.110000,-92.466389,Alder L.,50.11_-92.466


### Identifying common Lakes

In [43]:
merged_lake_name = bsm_lakes.merge(ahi_data, on=['lake_name'], how='inner')
print(len(merged_lake_name))
display(merged_lake_name)

4020


,lat_long_x,latitude_x,longitude_x,lake_name,latitude_y,longitude_y,lat_long_y
0,54.146_-85.04,54.14639,-85.03972,Pine L.,49.242500,-79.908333,49.242_-79.908
1,54.146_-85.04,54.14639,-85.03972,Pine L.,48.574722,-80.881667,48.575_-80.882
2,54.146_-85.04,54.14639,-85.03972,Pine L.,46.704444,-80.259444,46.704_-80.259
3,54.146_-85.04,54.14639,-85.03972,Pine L.,46.746389,-79.277778,46.746_-79.278
4,54.146_-85.04,54.14639,-85.03972,Pine L.,46.308889,-79.125833,46.309_-79.126
...,...,...,...,...,...,...,...
4015,44.434_-76.222,44.43417,-76.22167,South L.,44.434167,-76.221667,44.434_-76.222
4016,44.606_-76.669,44.60611,-76.66861,St. Andrew L.s,44.600000,-76.666667,44.6_-76.667
4017,44.425_-76.557,44.42472,-76.55722,Sydenham L.,44.424722,-76.557222,44.425_-76.557
4018,45.031_-77.431,45.03056,-77.43111,Weslemkoon L.,45.030556,-77.431111,45.031_-77.431


In [44]:
merged_lat_long = bsm_lakes.merge(ahi_data, on=['lat_long'], how='inner', suffixes=('_bsm', '_ahi'))
print(len(merged_lat_long))
display(merged_lat_long)
# merged_lat_long.to_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Ontario/common_lat_longs_ahi_bsm.csv', index=False)

673


,lat_long,latitude_bsm,longitude_bsm,lake_name_bsm,latitude_ahi,longitude_ahi,lake_name_ahi
0,54.165_-85.689,54.16500,-85.68917,Shamattawa L.,54.165000,-85.689167,Shamattawa L.
1,54.334_-85.014,54.33444,-85.01361,Spruce L.,54.334444,-85.013611,Spruce L.
2,52.579_-91.541,52.57861,-91.54139,Windigo L.,52.578611,-91.541389,Windigo L.
3,51.711_-94.485,51.71139,-94.48500,Cairns L.,51.711389,-94.485000,Cairns L.
4,54.521_-92.24,54.52056,-92.23972,Echoing L.,54.520556,-92.239722,Echoing L.
...,...,...,...,...,...,...,...
668,44.842_-77.252,44.84250,-77.25222,Skootamatta L.,44.842500,-77.252222,Skootamatta L.
669,44.434_-76.222,44.43417,-76.22167,South L.,44.434167,-76.221667,South L.
670,44.425_-76.557,44.42472,-76.55722,Sydenham L.,44.424722,-76.557222,Sydenham L.
671,45.031_-77.431,45.03056,-77.43111,Weslemkoon L.,45.030556,-77.431111,Weslemkoon L.


In [50]:
common_lake_names = np.unique([lake for lake in ahi_data['lake_name'].unique() if lake in bsm_lakes['lake_name'].unique()])

In [89]:
lake_name = 'Burt L.'
display(merged_lat_long.loc[merged_lat_long['lake_name_bsm'] == lake_name])
display(ahi_data.loc[ahi_data['lake_name'] == lake_name])
display(bsm_lakes.loc[bsm_lakes['lake_name'] == lake_name])

,lat_long,latitude_bsm,longitude_bsm,lake_name_bsm,latitude_ahi,longitude_ahi,lake_name_ahi


,latitude,longitude,lake_name,lat_long
4215,48.076667,-80.363056,Burt L.,48.077_-80.363


,lat_long,latitude,longitude,lake_name
237,48.297_-91.55,48.29694,-91.54972,Burt L.
360,48.293_-91.563,48.29252,-91.56278,Burt L.


In [63]:
missed_lakes = [lake for lake in common_lake_names if lake not in merged_lat_long['lake_name_bsm'].unique() or 
                lake not in merged_lat_long['lake_name_ahi'].unique()]
missed_lakes

['Aaron L.',
 'Ahmic L.',
 'Alexander L.',
 'Anishinabi L.',
 'Arbuckle L.',
 'Aubrey L.',
 'Barrel L.',
 'Basswood L.',
 'Bear L.',
 'Bee L.',
 'Berry L.',
 'Big Canon L.',
 'Big Moon L.',
 'Big Trout L.',
 'Biscotasi L.',
 'Blackstone L.',
 'Blair L.',
 'Bobs L.',
 'Bornite L.',
 'Bridge L.',
 'Bright L.',
 'Broadcast L.',
 'Bruce L.',
 'Brunswick L.',
 'Burt L.',
 'Calder L.',
 'Camel L.',
 'Canal L.',
 'Castle L.',
 'Cavell L.',
 'Chiniguchi L.',
 'Chipman L.',
 'Chub L.',
 'Cobble L.',
 'Cree L.',
 'Denyes L.',
 'Dilla L.',
 'Drag L.',
 'Duncan L.',
 'Dunstan L.',
 'Expanse L.',
 'Eye L.',
 'Fairbank L.',
 'Farrell L.',
 'Florence L.',
 'Flying Loon L.',
 'Fog L.',
 'Galloway L.',
 'Gary L.',
 'Gloucester Pl.',
 'Gordon L.',
 'Gould L.',
 'Grace L.',
 'Grassy L.',
 'Green L.',
 'Greenwood L.',
 'Griffin L.',
 'Hardiman L.',
 'Harris L.',
 'Hartman L.',
 'Heron L.',
 'Holden L.',
 'I-291, L.',
 'Jean L.',
 'Joyce L.',
 'Jubilee L.',
 'Kabinakagami L.',
 'Kabinakagamisis L.',
 'Kaga

In [64]:
len(missed_lakes)

146

# Thomas range shift

In [2]:
ahi_uniques = pd.read_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Analysis/Range_shifts/Ontario/Data/ahi_unique_lakes.csv')

display(ahi_uniques.head())

/home/arshad/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,notes,year,lat_long,lake_code,wby_name,wshed,latitude,longitude,alt_name,surface_area_ha,...,myoxocephalus_thompsonii,cottus_spp,esox_lucius_x_esox_americanus_vermiculatus,chrosomus_sppp,chrosomus_eos_x_chrosomus_neogaeus,chrosomus_eos_x_margariscus_nachtriebi,chrosomus_neogaeus_x_margariscus_nachtriebi,luxilus_cornutus_x_semotilus_atromaculatus,lepomis_sppp,lepomis_gibbosus_x_lepomis_macrochirus
0,NaN,1971.0,42.127_-83.12,L82001,Duff L.,2GH06,42.126667,-83.120000,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,1972.0,42.708_-81.17,L81002,Saunders P.,2GC02,42.707778,-81.169722,NaN,3.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,1972.0,42.712_-80.566,L84004,Deer Cr. Re.,2GC08,42.711667,-80.565833,NaN,26.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,1972.0,42.713_-81.119,L81018,Zavitz P.,2GC02,42.713333,-81.118889,NaN,2.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,1977.0,42.726_-80.318,L84012,Mathers P.,2GC07,42.726111,-80.318056,NaN,1.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
bsm_uniques = pd.read_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Analysis/Range_shifts/Ontario/Data/BSM_unique_lakes.csv')
display(bsm_uniques.head())

,Wby_LID,wby_name,year,lat_long,latitude,longitude,surface_area_ha,max_depth,mean_depth,sdf,...,lepomis_spp,labidesthes_sicculus,esox_americanus,oncorhynchus_tshawytscha,lepomis_gibbosus_x_lepomis_macrochirus,percopsidae_spp,esox_lucius_x_esox_masquinongy,dorosoma_cepedianum,notropis_stramineus,etheostoma_olmstedi
0,15-3523-55336,South Scot L.,2010,49.938_-95.058,49.93833,-95.05778,397,10.0,3.5,2.994879,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,15-3532-55170,Whitefish L.,2016,49.787_-95.035,49.78718,-95.03457,216,20.1,7.0,3.667483,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15-3543-56122,Wingiskus L.,2009,50.645_-95.061,50.64500,-95.06111,692,19.0,7.7,3.121802,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,15-3553-56171,Bee L.,2017,50.693_-95.061,50.69288,-95.06074,555,10.6,3.9,3.825031,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,15-3559-55281,Malachi L.,2010,49.89_-95.006,49.89028,-95.00611,1054,19.8,7.9,3.071787,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [124]:
# combined_ahi_bsm = pd.concat([ahi_uniques, bsm_uniques])
# combined_ahi_bsm.dropna(subset=['year'], inplace=True)
# combined_ahi_bsm.to_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Analysis/Range_shifts/Ontario/Data/combined_ahi_bsm_uniques.csv', 
#                         index=False)

In [4]:
non_species_cols = ['wby_name', 'year', 'lat_long', 'latitude', 'longitude', 'surface_area_ha', 'max_depth', 'mean_depth',
                    'conductivity_(us/cm)', 'tds_(mg/l)']
common_species = [col for col in bsm_uniques.columns if col in ahi_uniques.columns and col not in non_species_cols]
len(common_species)

91

In [5]:
threshold = 3

range_shifts = []

progress_bar =  tqdm(common_species)
for species in progress_bar:
    progress_bar.set_description("Processing %s" % species)
    # Subset Dataframe
    ahi_species = ahi_uniques[['latitude', 'longitude', 'year', 
                               species]].dropna(subset=[species]).drop_duplicates().sort_values(by=['latitude'], 
                                                                                                ascending = False)
    bsm_species = bsm_uniques[['latitude', 'longitude', 'year', 
                               species]].dropna(subset=[species]).drop_duplicates().sort_values(by=['latitude'], 
                                                                                                ascending = False)
    
    final_ahi_year = ahi_species['year'].max()
    final_bsm_year = bsm_species['year'].max()
    time_difference = final_bsm_year - final_ahi_year

    # Calculate threshold percentage
    rows_ahi = int(np.round(len(ahi_species) * threshold / 100))
    rows_bsm = int(np.round(len(bsm_species) * threshold / 100))

    # Subset ahi and bsm
    subset_ahi_species = ahi_species[:rows_ahi]
    subset_bsm_species = bsm_species[:rows_bsm]
    
    # Calculate average and median for lat
    
    avg_ahi_lat, median_ahi_lat = subset_ahi_species['latitude'].mean(), subset_ahi_species['latitude'].median()
    avg_bsm_lat, median_bsm_lat = subset_bsm_species['latitude'].mean(), subset_bsm_species['latitude'].median()

    # Range shifts
    
    abs_median_range_shift = (median_bsm_lat - median_ahi_lat) * 111
    abs_avg_range_shift = (avg_bsm_lat - avg_ahi_lat) * 111
    
    rel_avg_range_shift = abs_avg_range_shift / time_difference
    rel_median_range_shift = abs_median_range_shift / time_difference
    
    # Mannwhitney test
    
    mann_whitney = stats.mannwhitneyu(subset_ahi_species['latitude'], subset_bsm_species['latitude'])
    t_test = stats.ttest_ind(subset_ahi_species['latitude'], subset_bsm_species['latitude'])
    
    range_shifts.append([species, len(ahi_species), len(bsm_species), final_ahi_year, final_bsm_year, time_difference, 
                         threshold, 
                         avg_ahi_lat, avg_bsm_lat, median_ahi_lat, median_bsm_lat, 
                         abs_avg_range_shift, abs_median_range_shift, 
                         rel_avg_range_shift, rel_median_range_shift, 
                         mann_whitney.statistic, mann_whitney.pvalue, 
                         t_test.statistic, t_test.pvalue])

Processing cottoidea_spp:  19%|█▊        | 17/91 [00:00<00:02, 25.00it/s]         /home/arshad/miniconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3622: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/home/arshad/miniconda3/lib/python3.7/site-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
Processing cottus_ricei:  19%|█▊        | 17/91 [00:00<00:02, 25.00it/s]            /home/arshad/miniconda3/lib/python3.7/site-packages/scipy/stats/stats.py:6415: RuntimeWarning: divide by zero encountered in double_scalars
  z = (bigu - meanrank) / sd
Processing rhinichthys_cataractae:  25%|██▌       | 23/91 [00:01<00:03, 19.49it/s]/home/arshad/miniconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3622: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/home/arshad/miniconda3/lib/python3.7/site-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value enco

Processing semotilus_corporalis:  68%|██████▊   | 62/91 [00:02<00:00, 32.95it/s]                        /home/arshad/miniconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3622: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/home/arshad/miniconda3/lib/python3.7/site-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
Processing aplodinotus_grunniens:  74%|███████▎  | 67/91 [00:02<00:00, 34.62it/s]/home/arshad/miniconda3/lib/python3.7/site-packages/scipy/stats/stats.py:6415: RuntimeWarning: divide by zero encountered in double_scalars
  z = (bigu - meanrank) / sd
Processing oncorhynchus_mykiss:  74%|███████▎  | 67/91 [00:02<00:00, 34.62it/s]  /home/arshad/miniconda3/lib/python3.7/site-packages/scipy/stats/stats.py:6415: RuntimeWarning: divide by zero encountered in double_scalars
  z = (bigu - meanrank) / sd
Processing amia_calva:  74%|███████▎  | 67/91 [00:02<00:00, 34.62it/s]   

In [6]:
df_range_shifts = pd.DataFrame(range_shifts, 
                               columns = ['species', 'len_ahi', 'len_bsm', 'final_year_ahi', 'final_year_bsm', 
                                          'time_difference', 'threshold_%', 
                                           'avg_ahi_lat', 'avg_bsm_lat', 'median_ahi_lat', 'median_bsm_lat', 
                                           'avg_absolute_shift', 'median_absolute_shift', 
                                           'avg_relative_shift', 'median_relative_shift', 
                                          'mann_whitney_stats', 'mann_whitney_p', 
                                          't_test_stats', 't_test_p'])
display(df_range_shifts.head())

df_range_shifts.to_csv(f'/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Analysis/Range_shifts/Ontario/range_shifts_{today_date}.csv', 
                       index=False)

,species,len_ahi,len_bsm,final_year_ahi,final_year_bsm,time_difference,threshold_%,avg_ahi_lat,avg_bsm_lat,median_ahi_lat,median_bsm_lat,avg_absolute_shift,median_absolute_shift,avg_relative_shift,median_relative_shift,mann_whitney_stats,mann_whitney_p,t_test_stats,t_test_p
0,coregonus_artedi,2127,499,1986.0,2017.0,31.0,3,53.175608,53.009618,52.855139,52.909720,-18.424855,6.058491,-0.594350,0.195435,443.0,0.324105,0.712595,0.478250
1,notropis_atherinoides,258,209,1986.0,2017.0,31.0,3,54.334410,53.506908,54.215000,53.492255,-91.852643,-80.224695,-2.962988,-2.587893,3.0,0.004066,3.465852,0.004667
2,coregonus_clupeaformis,2089,448,1986.0,2017.0,31.0,3,53.503144,53.522569,53.461944,53.745000,2.156229,31.419216,0.069556,1.013523,394.5,0.420732,-0.096014,0.923769
3,percina_caprodes,603,225,1986.0,2017.0,31.0,3,53.617423,53.331549,53.639583,53.745000,-31.732043,11.701287,-1.023614,0.377461,51.0,0.243211,0.977920,0.338288
4,catostomus_catostomus,508,90,1986.0,2017.0,31.0,3,54.172796,54.333797,54.165000,54.334440,17.871052,18.807840,0.576486,0.606705,17.0,0.276808,-0.780700,0.446376


# Gains/Losses

In [105]:
df = pd.read_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Analysis/Range_shifts/Ontario/Data/common_ahi_bsm_lakes_unique_lakecodes.csv')
df.sort_values(by=['unique_lake_Id', 'year'], inplace=True)

df.head()

,unique_lake_Id,dataset,year,wby_name,latitude,longitude,surface_area_ha,max_depth,mean_depth,ichthyomyzon_fossor,...,hiodontidae_spp,moxostoma_carinatum,noturus_insignis,neogobius_melanostomus,moxostoma_erythrurum,cyprinella_spploptera,oncorhynchus_tshawytscha,percopsidae_spp,esox_lucius_x_esox_masquinongy,etheostoma_olmstedi
0,SL_001,AHI,1979,"20J-33, L.",49.904167,-80.033611,174.9,2.2,1.2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SL_001,BSM,2011,"20J-33, L.",49.904170,-80.033610,171.0,2.2,1.2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SL_001,BSM,2015,"20J-33, L.",49.904210,-80.033340,171.0,2.2,1.2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SL_002,AHI,1978,Abbess L.,48.680278,-91.682778,130.2,37.2,13.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,SL_002,BSM,2010,Abbess L.,48.680280,-91.682780,134.0,37.2,13.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [106]:
for col in ['latitude', 'longitude', 'surface_area_ha', 'max_depth', 'mean_depth']:
    df[col] = df.groupby(['unique_lake_Id'])[col].transform('mean')

df.fillna(0, inplace=True)

display(df.head(8))

,unique_lake_Id,dataset,year,wby_name,latitude,longitude,surface_area_ha,max_depth,mean_depth,ichthyomyzon_fossor,...,hiodontidae_spp,moxostoma_carinatum,noturus_insignis,neogobius_melanostomus,moxostoma_erythrurum,cyprinella_spploptera,oncorhynchus_tshawytscha,percopsidae_spp,esox_lucius_x_esox_masquinongy,etheostoma_olmstedi
0,SL_001,AHI,1979,"20J-33, L.",49.904182,-80.033520,172.300000,2.2,1.2,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,SL_001,BSM,2011,"20J-33, L.",49.904182,-80.033520,172.300000,2.2,1.2,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,SL_001,BSM,2015,"20J-33, L.",49.904182,-80.033520,172.300000,2.2,1.2,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,SL_002,AHI,1978,Abbess L.,48.680109,-91.683133,132.733333,37.2,13.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,SL_002,BSM,2010,Abbess L.,48.680109,-91.683133,132.733333,37.2,13.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,SL_002,BSM,2015,Abbess L.,48.680109,-91.683133,132.733333,37.2,13.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,SL_003,AHI,1968,"Abitibi, L.",48.658238,-79.844917,86913.200000,15.3,3.5,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,SL_003,BSM,2010,"Abitibi, L.",48.658238,-79.844917,86913.200000,15.3,3.5,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [110]:
bsm_only = df.loc[df['dataset'] == 'BSM']
bsm_only = bsm_only[bsm_only.duplicated(['unique_lake_Id'], keep=False)]
bsm_only

,unique_lake_Id,dataset,year,wby_name,latitude,longitude,surface_area_ha,max_depth,mean_depth,ichthyomyzon_fossor,...,hiodontidae_spp,moxostoma_carinatum,noturus_insignis,neogobius_melanostomus,moxostoma_erythrurum,cyprinella_spploptera,oncorhynchus_tshawytscha,percopsidae_spp,esox_lucius_x_esox_masquinongy,etheostoma_olmstedi
1,SL_001,BSM,2011,"20J-33, L.",49.904182,-80.033520,172.300000,2.200000,1.200000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,SL_001,BSM,2015,"20J-33, L.",49.904182,-80.033520,172.300000,2.200000,1.200000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,SL_002,BSM,2010,Abbess L.,48.680109,-91.683133,132.733333,37.200000,13.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,SL_002,BSM,2015,Abbess L.,48.680109,-91.683133,132.733333,37.200000,13.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,SL_003,BSM,2010,"Abitibi, L.",48.658238,-79.844917,86913.200000,15.300000,3.500000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1856,SL_695,BSM,2013,"des Iles, La.",49.202234,-89.606517,1538.900000,29.033333,5.833333,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1858,SL_696,BSM,2008,"des Mille Lacs, La.",48.858289,-90.511126,24378.266667,24.400000,6.800000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1859,SL_696,BSM,2013,"des Mille Lacs, La.",48.858289,-90.511126,24378.266667,24.400000,6.800000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1861,SL_697,BSM,2008,"of Bays, L.",45.254784,-79.012100,6810.033333,70.100000,22.200000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [111]:
# lake_groups = df.groupby('unique_lake_Id')

lake_groups = bsm_only.groupby('unique_lake_Id')

df_first_last = (pd.concat([lake_groups.head(1), 
                            lake_groups.tail(1)])).drop_duplicates().sort_values(['unique_lake_Id', 'year']).reset_index(drop=True)

In [48]:
df_first_last = pd.read_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Analysis/Range_shifts/Ontario/Data/Extras/first_last_common_lakes.csv')
df_first_last.head()

,unique_lake_Id,dataset,year,wby_name,latitude,longitude,surface_area_ha,max_depth,mean_depth,ichthyomyzon_fossor,...,hiodontidae_spp,moxostoma_carinatum,noturus_insignis,neogobius_melanostomus,moxostoma_erythrurum,cyprinella_spploptera,oncorhynchus_tshawytscha,percopsidae_spp,esox_lucius_x_esox_masquinongy,etheostoma_olmstedi
0,SL_001,AHI,1979,"20J-33, L.",49.904182,-80.033520,172.300000,2.2,1.2,0,...,0,0,0,0,0,0,0,0,0,0
1,SL_001,BSM,2015,"20J-33, L.",49.904182,-80.033520,172.300000,2.2,1.2,0,...,0,0,0,0,0,0,0,0,0,0
2,SL_002,AHI,1978,Abbess L.,48.680109,-91.683133,132.733333,37.2,13.0,0,...,0,0,0,0,0,0,0,0,0,0
3,SL_002,BSM,2015,Abbess L.,48.680109,-91.683133,132.733333,37.2,13.0,0,...,0,0,0,0,0,0,0,0,0,0
4,SL_003,AHI,1968,"Abitibi, L.",48.658238,-79.844917,86913.200000,15.3,3.5,0,...,0,0,0,0,0,0,0,0,0,0


In [112]:
unique_lake_codes = df_first_last['unique_lake_Id'].unique()

species = df_first_last.columns[9:]

gains_losses = []

progress_bar =  tqdm(unique_lake_codes)

for lakecode in progress_bar:
    progress_bar.set_description("Processing %s" % lakecode)
    df_lake = df_first_last.loc[df_first_last['unique_lake_Id'] == lakecode].sort_values(by=['unique_lake_Id', 'year'])
    df_lake.drop(columns=['dataset', 'year'], inplace=True)
    temp_df = df_lake[df_lake.columns[:7]][-1:]
    
    for specie in species:
        ahi_specie = df_lake.iloc[0][specie] # ahi
        bsm_specie = df_lake.iloc[1][specie] # bsm
        temp_df[specie] = bsm_specie - ahi_specie
        
    gains_losses.append(temp_df)

Processing SL_697: 100%|██████████| 468/468 [01:00<00:00,  7.73it/s]


In [113]:
analysis_file_path = '/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Analysis/Range_shifts/Ontario/'

df_gains_losses = pd.concat(gains_losses)
display(df_gains_losses.head())

df_gains_losses.to_csv(f'{analysis_file_path}gains_losses_lake_focused_bsm_only_{today_date}.csv', index=False)

transposed_df_gains_losses = df_gains_losses.drop(columns=['unique_lake_Id', 'latitude', 'longitude', 'surface_area_ha', 
                                                           'max_depth', 'mean_depth']).set_index('wby_name').T
display(transposed_df_gains_losses.head())

transposed_df_gains_losses.to_csv(f'{analysis_file_path}gains_losses_species_focused_bsm_only_{today_date}.csv', index=False)

,unique_lake_Id,wby_name,latitude,longitude,surface_area_ha,max_depth,mean_depth,ichthyomyzon_fossor,ichthyomyzon_unicuspis,petromyzon_marinus,...,hiodontidae_spp,moxostoma_carinatum,noturus_insignis,neogobius_melanostomus,moxostoma_erythrurum,cyprinella_spploptera,oncorhynchus_tshawytscha,percopsidae_spp,esox_lucius_x_esox_masquinongy,etheostoma_olmstedi
1,SL_001,"20J-33, L.",49.904182,-80.033520,172.300000,2.2,1.2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,SL_002,Abbess L.,48.680109,-91.683133,132.733333,37.2,13.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,SL_003,"Abitibi, L.",48.658238,-79.844917,86913.200000,15.3,3.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,SL_004,Achigan L.,46.918004,-84.196449,286.533333,44.2,12.4,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,SL_006,Adelaide L.,47.059883,-84.307226,131.766667,47.2,12.2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


wby_name,"20J-33, L.",Abbess L.,"Abitibi, L.",Achigan L.,Adelaide L.,Agnes L.,Agnew L.,Aileen L.,Amik L.,Andy L.,...,Winnange L.,Wintering L.,Wintering L.,Woodosgoon L.,Wyder L.,Young L.,Zadi L.,"des Iles, La.","des Mille Lacs, La.","of Bays, L."
ichthyomyzon_fossor,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ichthyomyzon_unicuspis,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
petromyzon_marinus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
acipenser_fulvescens,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
lepisosteus_osseus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Climate Warming Rates

In [4]:
gains_losses = pd.read_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Analysis/Range_shifts/Ontario/Thomas_jan_05/Gains&LossesCorrelations.csv')
print(len(gains_losses))
gains_losses.head()

696


,unique_lake_Id,wby_name,latitude,longitude,surface_area_ha,max_depth,mean_depth,LakeSUMChanges,Centrarchidae,Ictalurid,...,perca_flavescens_yellow_perch,percina_caprodes_common_logperch,percopsis_omiscomaycus_trout-perch,pimephales_notatus_bluntnose_minnow,pimephales_promelas_fathead_minnow,pungitius_pungitius_ninespine_stickleback,salvelinus_fontinalis_brook_trout,salvelinus_namaycush_lake_trout,sander_vitreus_walleye,semotilus_atromaculatus_creek_chub
0,SL_001,"20J-33, L.",49.904182,-80.033520,172.300000,2.2,1.2,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,SL_002,Abbess L.,48.680109,-91.683133,132.733333,37.2,13.0,-2,1,0,...,0,0,0,0,0,0,0,0,0,0
2,SL_003,"Abitibi, L.",48.658238,-79.844917,86913.200000,15.3,3.5,0,0,1,...,0,-1,0,0,0,-1,0,0,0,0
3,SL_004,Achigan L.,46.918004,-84.196449,286.533333,44.2,12.4,3,0,0,...,0,0,0,0,0,0,1,0,0,0
4,SL_005,Addie L.,48.166390,-90.431111,115.550000,3.7,2.1,4,1,0,...,1,0,0,0,0,0,0,0,1,0


In [23]:
climate_warming_df = pd.read_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Analysis/Range_shifts/Ontario/Climate_Warming_and_Warming_Rate.csv')
print(len(climate_warming_df))
climate_warming_df.drop(columns = ['notes', 'year', 'wshed', 'lat_long', 'lake_code'], 
                        inplace=True)
climate_warming_df = climate_warming_df.loc[climate_warming_df['wby_name'].isin(gains_losses['wby_name'].unique())]
climate_warming_df.head()

10732


,wby_name,latitude,longitude,slpTrends,voccMag
8,Bear L.,49.206389,-81.287778,0.032995,230.164627
13,Horseshoe L.,49.224722,-81.300833,0.032995,230.164627
29,Clear L.,48.317500,-81.295000,0.032834,65.986847
30,Griffin L.,48.912500,-82.388333,0.030644,63.542948
41,South L.,49.455000,-84.596667,0.022314,28.005911


In [25]:
gains_losses_climate_warming = pd.merge(gains_losses, climate_warming_df, on='wby_name', how='left')
gains_losses_climate_warming.drop_duplicates(inplace=True)
print(len(gains_losses_climate_warming))
gains_losses_climate_warming.head()

2652


,unique_lake_Id,wby_name,latitude_x,longitude_x,surface_area_ha,max_depth,mean_depth,LakeSUMChanges,Centrarchidae,Ictalurid,...,pimephales_promelas_fathead_minnow,pungitius_pungitius_ninespine_stickleback,salvelinus_fontinalis_brook_trout,salvelinus_namaycush_lake_trout,sander_vitreus_walleye,semotilus_atromaculatus_creek_chub,latitude_y,longitude_y,slpTrends,voccMag
0,SL_001,"20J-33, L.",49.904182,-80.033520,172.300000,2.2,1.2,1,0,0,...,0,0,0,0,0,0,49.904167,-80.033611,0.035856,9.207911
1,SL_001,"20J-33, L.",49.904182,-80.033520,172.300000,2.2,1.2,1,0,0,...,0,0,0,0,0,0,49.904210,-80.033340,0.035856,9.207911
2,SL_002,Abbess L.,48.680109,-91.683133,132.733333,37.2,13.0,-2,1,0,...,0,0,0,0,0,0,48.680278,-91.682778,0.011987,0.235912
3,SL_002,Abbess L.,48.680109,-91.683133,132.733333,37.2,13.0,-2,1,0,...,0,0,0,0,0,0,48.679770,-91.683840,0.011987,0.235912
4,SL_003,"Abitibi, L.",48.658238,-79.844917,86913.200000,15.3,3.5,0,0,1,...,0,-1,0,0,0,0,48.658333,-79.845000,0.034649,1.398136


In [26]:
gains_losses_climate_warming.to_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Analysis/Range_shifts/Ontario/Thomas_jan_05/gains_losses_climate_warming.csv', 
                                    index=False)

In [31]:
gains_losses_climate_warming = pd.read_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Analysis/Range_shifts/Ontario/Thomas_jan_05/gains_losses_climate_warming.csv')
gains_losses_climate_warming.dropna(how='all', inplace=True)
gains_losses_climate_warming.to_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Analysis/Range_shifts/Ontario/Thomas_jan_05/gains_losses_climate_warming.csv', 
                                    index=False)